Install


In [2]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip uninstall stable-baselines
!pip install stable-baselines[mpi]
!pip install gym[mujoco]

!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym > /dev/null 2>&1
!pip install pyglet

!pip install pybullet

!pip install tensorboard

Uninstalling stable-baselines-2.9.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/stable_baselines-2.9.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/stable_baselines/*
Proceed (y/n)? y
  Successfully uninstalled stable-baselines-2.9.0
  Using cached https://files.pythonhosted.org/packages/c0/05/f6651855083020c0363acf483450c23e38d96f5c18bec8bded113d528da5/stable_baselines-2.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cf/8c/64e0630b3d450244feef0688d90eab2448631e40ba6bdbd90a70b84898e7/mujoco-py-1.50.1.68.tar.gz
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-w_j3endk/mujoco-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-w_j3endk/mujoco-py/setup.py'"'"';f=getattr(tokenize, '"'"'op

Imports


In [3]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
#import tensorflow as tf
import numpy as np
#import random
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib inline
#import math
#import glob
#import io
import base64
#import numpy as np
#from IPython.display import HTML

#from IPython import display as ipythondisplay

import pybullet_envs

from stable_baselines import TD3
from stable_baselines.td3.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

%load_ext tensorboard

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


Help functions to save video

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
n_steps = 0

def callback(_locals, _globals):
    global n_steps
    # Save model every 1000 calls
    if (n_steps + 1) % 1000 == 0:
        model.save(dir_model)
    n_steps += 1

    return True
    

def wrap_env(env):
  env = Monitor(env, dir + 'video', force=True)
  return env

Mounting Google Drive to save model and videos

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Definition Strings

In [0]:

#envStrings
envPendulum = 'Pendulum-v0'
envCheetah = 'HalfCheetah'
envAnt = 'AntBulletEnv-v0'
envHuman = 'HumanoidBulletEnv-v0'

model_string = envPendulum

dir = '/content/drive/My Drive/DLSeminar/models/' + model_string + "/"


dir_logs = dir + 'logs/'

dir_model = dir + model_string

model_steps = 50000

Impl

In [0]:
env = gym.make(model_string)
env = gym.wrappers.Monitor(env, dir + '/video2', force=True)
env = wrap_env(env)

env = DummyVecEnv([lambda: env])
n_actions = env.action_space.shape[-1]
#print(env.action_space.shape[0])
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))



#model = TD3(MlpPolicy, env, action_noise=action_noise, verbose=1, tensorboard_log=dir_logs, learning_rate=0.3)
#model.learn(total_timesteps=model_steps, log_interval=10, callback=callback)
#model.save(dir_model)




#del model # remove to demonstrate saving and loading

model = TD3.load(dir_model, env)
model.learn(total_timesteps=model_steps, log_interval=1, callback=callback)

#Launch TensorBoard after learning
#%tensorboard --logdir /content/drive/My\ Drive/DLSeminar/models/AntBulletEnv-v0/logs/
%tensorboard --logdir /content/drive/My\ Drive/DLSeminar/models/Pendulum-v0/logs/
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
            
env.close()





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





---------------------------------------
| current_lr              | 0.3       |
| episodes                | 10        |
| fps                     | 76        |
| mean 100 episode reward | -1.53e+03 |
| n_updates               | 1600      |
| qf1_loss                | 20.516052 |
| qf2_loss                | 16.865673 |
| time_elapsed            | 23        |
| total timesteps         | 1800      |
---------------------------------------
---------------------------------------
| current_lr              | 0.3       |
| episodes                | 20        |
| fps                     | 119       |
| mean 100 episode reward | -1.48e+03 |
| n_updates               | 3600      |
| qf1_loss           

In [0]:
%tensorboard --logdir /content/drive/My\ Drive/DLSeminar/models/Pendulum-v0/logs/